In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import glob
from tqdm import tqdm
import h5py
from PIL import Image
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
batch_size = 64
train_dir = os.path.join("resized_images", "train")

datagen = ImageDataGenerator()
train_data = datagen.flow_from_directory(batch_size=batch_size,
                                         directory=train_dir,
                                         image_size=(224,224),
                                         shuffle=True,
                                         class_mode='categorical')
# Confirm iterator works
batchX, batchY = train_data.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

TypeError: flow_from_directory() got an unexpected keyword argument 'image_size'

In [1]:
# for label in tqdm(range(0,1)):
#     np_images = []
#     np_labels = []
#     images = glob.glob(f"resized_images/train/{label:02d}/*.jpg")

#     for i in range(len(images)):
#         image = Image.open(images[i])
#         np_image = np.asarray(image)
#         np_images.append(np_image/255)
#         np_labels.append(label)

#     data_x = np.asarray(np_images)
#     data_y = np.asarray(np_labels)
#     hf = h5py.File(f'pickles/data_{label}.h5', 'w')
#     hf.create_dataset(f'x', data=data_x,  compression='gzip', compression_opts=9)
#     hf.create_dataset(f'y', data=data_y)
#     hf.close()


In [3]:
def get_model():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2, padding='same', activation = 'relu', input_shape = (224, 224, 3))) 
    model.add(tf.keras.layers.MaxPooling2D(pool_size = 2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(filters = 16, kernel_size = 2, padding = 'same', activation = 'relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size = 2))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(42, activation = 'softmax'))

    # Take a look at the model summary
    model.summary()
    return model

In [4]:
model = get_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 16)      2064      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 56, 56, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0

In [5]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [8]:
model.fit(
    x=batchX,
    y=batchY,
    epochs=10)

Epoch 1/10


ValueError: in user code:

    /Users/bhanuka/.pyenv/versions/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Users/bhanuka/.pyenv/versions/tensorflow/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/bhanuka/.pyenv/versions/tensorflow/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/bhanuka/.pyenv/versions/tensorflow/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/bhanuka/.pyenv/versions/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
  

In [ ]:
model.evaluate(
    x=data_x,
    y=data_y
)